<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Notebook</p></tp>
            <tp><p style="font-size:150%;text-align:center">Lógica de predicados</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

## Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar en Python la lógica proposicional y el cálculo $\lambda$ usando el módulo de lógica de la librería `nlkt`. También veremos una manera de responder preguntas sobre un texto de manera automática usando una base de conocimiento. 

[Ir al ejercicio 1](#ej1)

## Dependencias

Al iniciar el notebook o reiniciar el kerner, se pueden cargar todas las dependencias de este notebook al correr las siguientes celdas. Este también es el lugar para instalar las dependencias que podrían hacer falta.

**De Python:**

In [ ]:
from nltk.sem.logic import LogicParser
from nltk import load_parser
from nltk.grammar import FeatureGrammar
from nltk.parse import FeatureEarleyChartParser
from nltk.tree import Tree
from itertools import product

**Del notebook:**

In [ ]:
import utils
from logica import LPQuery
from parseMod import Modelo
from logUtils import Ytoria
from resPreg import pipeline_responder_preguntas

## Secciones

Desarrollaremos la explicación en las siguientes secciones:

* [Fórmulas de la lógica de predicados.](#predicados)
* [Fórmulas del cálculo $\lambda$.](#calculo)
* [Gramática lógica.](#gramatica)
* [Ejemplo con el mundo de los bloques.](#bloques)
* [Representación de preguntas.](#preguntas)
* [Resolución de preguntas.](#answer)

## Fórmulas de la lógica de predicados <a class="anchor" id="predicados"></a>

([Volver al inicio](#inicio))

La librería `nltk` tiene un módulo mediante el cual se pueden representar las fórmulas de la lógica de predicados como objetos ([ver módulo nltk.sem.logic](https://www.nltk.org/api/nltk.sem.logic.html#module-nltk.sem.logic) y ver la explicación [aquí](https://www.nltk.org/book_1ed/ch10.html)). Más aún, `nltk` tiene implementado un parser que nos sirve para leer una fórmula como cadena y representarla como un objeto.

In [ ]:
# Cargamos el parser
lp = LogicParser()

Veamos esta funcionalidad para el caso de las fórmulas atómicas, los conectivos lógicos y las fórmulas cuantificadas de la lógica de predicados.

**Caso 1: Fórmulas atómicas**

Comenzamos por considerar las fórmulas atómicas, las cuales se componen de un predicado aplicado a una o varias constantes (dependiendo de la aridad del predicado). 

In [ ]:
# Escribimos la fórmula
formula_cadena = r'AMAR(pedro, x)'
#                ^
#                observe el "r"
# usamos "r" al comienzo de la cadena para poder usar
# símbolos reservados, como el "\"

# Creamos el objeto mediante el parser
formula_objeto = lp.parse(formula_cadena)
print(formula_objeto)

Una fórmula atómica como objeto tiene algunos atributos, como sus argumentos: 

In [ ]:
print(formula_objeto.args)

O como sus variables libres

In [ ]:
print(formula_objeto.free())

Revise el [módulo de logica](https://www.nltk.org/api/nltk.sem.logic.html#module-nltk.sem.logic) para examinar los demás atributos de las fórmulas atómicas.

**Caso 2: Conectivos lógicos**

Los conectivos lógicos deben escribirse de la siguiente manera:

| Nombre | nltk |
| :---: | :---: |
| Negación | - |
| Y | &|
| O | \| |
| Implicación | -> |
| Equivalencia | <-> |

In [ ]:
formula_cadena = r'CAMINA(pedro) & MIRA(pedro, maria)'
formula_objeto = lp.parse(formula_cadena)
print(formula_objeto)

Observe que el parser asume que unos operadores preceden a otros, de tal manera que no hay riesgo de ambigüedad en las expresiones. Por ejemplo, observe que la siguiente fórmula es ambigua, pero el parser asume que "&" tiene precedencia sobre "|":

In [ ]:
formula_cadena = r'CAMINA(pedro) & MIRA(pedro,maria) | CANTAR(PEDRO)'
formula_objeto = lp.parse(formula_cadena)
print(formula_objeto)

**Caso 3: Cuantificadores**

El cuantificador universal y el existencial se incorporan fácilmente:

In [ ]:
formula_cadena = r'exists x.CAMINA(x)'
#                          ^
#                      observe el punto
formula_objeto = lp.parse(formula_cadena)
print(formula_objeto)

In [ ]:
formula_cadena = r'all x.CAMINA(x)'
#                       ^
#                   observe el punto
formula_objeto = lp.parse(formula_cadena)
print(formula_objeto)

La variable del cuantificador se puede acceder fácilmente mediante el atributo `variable`:

In [ ]:
formula_objeto.variable

Y la fórmula en el rango del cuantificador se puede acceder mediante el atributo `term`:

In [ ]:
print(formula_objeto.term)

<a class="anchor" id="ej1"></a>**Ejercicio 1:** 

([Próximo ejercicio](#ej2))

Considere el *mundo de los bloques*, el cual consiste en un conjunto de bloques bidimensionales de forma cuadrada, con un color y con una letra única para cada bloque. Ellos se encuentran encima de una mesa y pueden estar apilados unos encima de otros.

<img src="./imagenes/bloques2.png" width="auto"/>

Cree las fórmulas como objetos correspondientes a las siguientes expresiones:

* $A$ no está a la izquierda de $B$ y $C$ es morado.
* Si $C$ es morado, entonces no es verde.
* No es cierto que todo bloque está debajo de $C$.

**Nota:** use la expresión `bloque_A` para referirse al bloque $A$ y así para los tres bloques. Esto debemos hacerlo porque si usamos `A`, el parser pensará que es una variable y lo que queremos es una constante.

In [ ]:
f1_ = ... # AQUÍ LA FÓRMULA PARA LA PRIMERA EXPRESIÓN
f1 = lp.parse(f1_)
print('La fórmula correspondiente a')
print('\n\t 𝐴  no está a la izquierda de 𝐵 y 𝐶 es morado.')
print('es:')
print('\n\t', f1)

f2_ = ... # AQUÍ LA FÓRMULA PARA LA SEGUNDA EXPRESIÓN
f2 = lp.parse(f2_)
print('\nLa fórmula correspondiente a')
print('\n\t Si  𝐶  es morado, entonces no es verde.')
print('es:')
print('\n\t', f2)

f3_ = ... # AQUÍ LA FÓRMULA PARA LA TERCERA EXPRESIÓN
f3 = lp.parse(f3_)
print('\nLa fórmula correspondiente a')
print('\n\t No es cierto que todo bloque está debajo de  𝐶 .')
print('es:')
print('\n\t', f2)

---

## Fórmulas del cálculo $\lambda$ <a class="anchor" id="calculo"></a>

([Volver al inicio](#inicio))

Recordemos que el cálculo $\lambda$ es muy útil, pues sirve para crear funciones anónimas que pueden operarse entre sí. El operador $\lambda$ funciona como un cuantificador, pero no se escribe "lambda", sino solo "\\":

In [ ]:
formula_cadena1 = r'\x.CAMINAR(x)'
formula_objeto1 = lp.parse(formula_cadena1)
print(formula_objeto1)

Esta expresión representa una función que toma constantes y devuelve una fórmula. Por ejemplo, podemos aplicarle la constante `pedro` a la anterior expresión para obtener la fórmula `CAMINAR(pedro)`:

In [ ]:
formula_cadena2 = fr'{formula_objeto1}(pedro)'
formula_objeto2 = lp.parse(formula_cadena2)
print(formula_objeto2)

Observe que debemos simplificar la expresión mediante el método `simplify()` para obtener el resultado deseado:

In [ ]:
print(formula_objeto2.simplify())

Un ejemplo más complicado es el siguiente. Considere la representación lógica de las siguientes expresiones (como se definió en las diapositivas de clase):

* "hombre" $\Rightarrow$ $\lambda x~HOMBRE(x)$
* "camina" $\Rightarrow$ $\lambda x~CAMINAR(x)$ 
* "un" $\Rightarrow$ $\lambda X~\lambda Y~\exists y\left(X(y)\land Y(y)\right)$

Creamos las fórmulas para cada expresión:

In [ ]:
hombre_ = r'\x.HOMBRE(x)'
camina_ = r'\x.CAMINAR(x)'
un_ = r'\X.(\Y.(exists y.(X(y) & Y(y))))'
hombre = lp.parse(hombre_)
camina = lp.parse(camina_)
un = lp.parse(un_)

Aplicamos la expresión "hombre" a la expresión "un" para obtener "un hombre":

In [ ]:
un_hombre_ = fr'({un})({hombre})'
un_hombre = lp.parse(un_hombre_).simplify()
print(un_hombre)

Ahora aplicamos la expresión "camina" a la expresión "un hombre" para obtener "un hombre camina":

In [ ]:
un_hombre_camina_ = fr'({un_hombre})({camina})'
un_hombre_camina = lp.parse(un_hombre_camina_).simplify()
print(un_hombre_camina)

<a class="anchor" id="ej2"></a>**Ejercicio 2:** 

([Anterior ejercicio](#ej1)) ([Próximo ejercicio](#ej3))

A partir de la representación de las expresiones "pedro", "ama" y "maria" que vimos en las diapositivas de clase, realice un procedimiento similar al que acabamos de usar para obtener la representación de "pedro ama maria". Observe que en este caso **debemos obtener primero la expresión "ama maria"** para luego obtener la expresión "pedro ama maria", que debe resultar en "AMAR(pedro,maria)".  

Recuerde que las representaciones léxicas son:

* "pedro" $\Rightarrow$ $\lambda X~X(pedro)$
* "maria" $\Rightarrow$ $\lambda X~X(maria)$
* "amar" $\Rightarrow$ $\lambda P(\lambda x(P(\lambda y~AMAR(x,y))))$ 

***

## Gramática lógica <a class="anchor" id="gramatica"></a>

([Volver al inicio](#inicio))

¿Cómo sabemos que debemos encontrar primero la expresión "ama maria" antes que la expresión "pedro ama"? La respuesta la da la sintaxis. En el árbol de la derecha, vemos que las hojas "ama" y "maria" se unen primero y que su resultado se une con "pedro" para obtener la oración completa. También podemos ver en el árbol de la izquierda que el orden de aplicación es diferente. Primero se unen "un" y "hombre" para luego unirse con "camina" y así formar la oración completa:

<img src="./imagenes/arboles.png" width="400"/>

La simplificación de las expresiones se realiza de acuerdo a la manera en que los nodos se unen en el árbol de análisis de la expresión. Observe que estamos juntando dos ideas: cálculo $\lambda$ con gramáticas. Esto es lo que se conoce como una gramática lógica.

En una gramática lógica, debemos representar las expresiones de acuerdo a su fórma lógica y unirlas de acuerdo a lo que nos diga su árbol de análisis. Resulta que la librería `nltk` nos da esta funcionalidad, gracias al uso de gramáticas independientes del contexto con características gramaticales.

En efecto, una de las características de las expresiones puede ser su representación semántica. De esta manera, cada expresión puede asignarse a una fórmula del cálculo $\lambda$. Adicionalmente, las reglas gramaticales pueden decirnos la manera de obtener la fórmula correspondiente a la madre con respecto a las fórmulas de sus hijos.

Ilustremos esta idea mediante un ejemplo sencillo. Considere la siguiente gramática:

In [ ]:
# Definimos la gramática
reglas1 = r"""
% start O
#############################
# Reglas gramaticales
#############################

# La oración es un sintagma nominal (SN) seguido de un verbo (V)
O[SEM=<?sn(?v)>] -> SN[SEM=?sn] V[SEM=?v] 
# Un sintagma nominal (SN) puede ser un término (T)
SN[SEM=?t] -> T[SEM=?t]
# Un verbo (V) puede ser un verbo intansitivo (VI)
V[SEM=?v] -> VI[SEM=?v]

#############################
# Reglas léxicas
#############################

T[SEM=<\X.X(pedro)>] -> 'pedro'
VI[SEM=<\x.CAMINAR(x)>] -> 'camina'
"""

# Instanciamos el objeto
gramatica = FeatureGrammar.fromstring(reglas1)

# Instanciamos el parser
parser = FeatureEarleyChartParser(gramatica)

# Procesamos la oración
oracion = 'pedro camina'.split()
tree1 = utils.parsear(oracion, parser, verbose=True)
print('El árbol bidimensional es:')
tree1

<a class="anchor" id="ej3"></a>**Ejercicio 3:** 

([Anterior ejercicio](#ej2)) ([Próximo ejercicio](#ej4))

Modifique la gramática `reglas1` para poder encontrar la representación de "un hombre camina".

In [ ]:
# Definimos la gramática
reglas1 = r"""
% start O
#############################
# Reglas gramaticales
#############################

# La oración es un sintagma nominal (SN) seguido de un verbo (V)
O[SEM=<?sn(?v)>] -> SN[SEM=?sn] V[SEM=?v] 
# Un sintagma nominal (SN) puede ser un término (T)
SN[SEM=?t] -> T[SEM=?t]
# Un verbo (V) puede ser un verbo intansitivo (VI)
V[SEM=?v] -> VI[SEM=?v]
# Un sintagma nominal (SN) puede ser un determinante (D) seguido de un sustativo (N)
>>>>>>>>>>> AQUI LA EXPRESIÓN DE LA GRAMÁTICA CON CARACTERÍSTICAS SEMÁNTICAS <<<<<<<<<<

#############################
# Reglas léxicas
#############################

T[SEM=<\X.X(pedro)>] -> 'pedro'
VI[SEM=<\x.CAMINAR(x)>] -> 'camina'
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE HOMBRE <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE UN <<<<<<<<<<
"""

# Instanciamos el objeto
gramatica = FeatureGrammar.fromstring(reglas1)

# Instanciamos el parser
parser = FeatureEarleyChartParser(gramatica)

# Procesamos la oración
oracion = 'un hombre camina'.split()
tree1 = utils.parsear(oracion, parser, verbose=True)
print('El árbol bidimensional es:')
tree1

---

<a class="anchor" id="ej4"></a>**Ejercicio 4:** 

([Anterior ejercicio](#ej3)) ([Próximo ejercicio](#ej5))

Modifique la gramática `reglas1` para poder encontrar la representación de "pedro ama maria".

In [ ]:
# Definimos la gramática
reglas1 = r"""
% start O
#############################
# Reglas gramaticales
#############################

# La oración es un sintagma nominal (SN) seguido de un verbo (V)
O[SEM=<?v(?sn)>] -> SN[SEM=?sn] V[SEM=?v] 
# Un sintagma nominal (SN) puede ser un término (T)
SN[SEM=?t] -> T[SEM=?t]
# Un verbo (V) puede ser un verbo intansitivo (VI)
V[SEM=?v] -> VI[SEM=?v]
# Un sintagma nominal (SN) puede ser un determinante (D) seguido de un sustativo (N)
>>>>>>>>>>> AQUI LA EXPRESIÓN DE LA GRAMÁTICA CON CARACTERÍSTICAS SEMÁNTICAS <<<<<<<<<<
# Un verbo (V) puede ser un verbo tansitivo (VT) seguido de un sintagma nominal (SN)
>>>>>>>>>>> AQUI LA EXPRESIÓN DE LA GRAMÁTICA CON CARACTERÍSTICAS SEMÁNTICAS <<<<<<<<<<

#############################
# Reglas léxicas
#############################

T[SEM=<pedro>] -> 'pedro'
VI[SEM=<\x.CAMINAR(x)>] -> 'camina'
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE HOMBRE <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE UN <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE MARIA <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE AMAR <<<<<<<<<<
"""

# Instanciamos el objeto
gramatica = FeatureGrammar.fromstring(reglas1)

# Instanciamos el parser
parser = FeatureEarleyChartParser(gramatica)

# Procesamos la oración
oracion = 'pedro ama maria'.split()
tree1 = utils.parsear(oracion, parser, verbose=True)
print('El árbol bidimensional es:')
tree1

---

## Ejemplo con el mundo de los bloques <a class="anchor" id="bloques"></a>

([Volver al inicio](#inicio))

El mundo de los bloques es suficientemente sencillo para nuestros propósitos. Vamos a crear la gramática que procesará las oraciones en español y devolverá fórmulas de la lógica de predicados que representan situaciones en el mundo de los bloques. Una opción es la siguiente gramática:

In [ ]:
# Definimos la gramática para el mundo de los bloques
reglas_bloques = r"""
% start O
#############################
# Reglas gramaticales
#############################

# La oración es un bloque (BL) seguido de 'está' y un vector (VE)
O[SEM=<?ve(?b)>] -> BL[SEM=?b] 'está' VE[SEM=?ve]

# Un vector (VE) es una dirección (D) seguida de un bloque (BL)
VE[SEM=<?d(?b)>] -> D[SEM=?d] BL[SEM=?b]

#############################
# Reglas léxicas
#############################

BL[SEM=<bloque_A>] -> 'A'
BL[SEM=<bloque_B>] -> 'B'
D[SEM=<\y.(\x.DERECHA(x,y))>] -> 'a' 'la' 'derecha' 'de'
"""

# Instanciamos el objeto
gramatica_bloques = FeatureGrammar.fromstring(reglas_bloques)

# Instanciamos el parser
parser_bloques = FeatureEarleyChartParser(gramatica_bloques)

Veámos su funcionamiento, aplicado sobre la oración:

* "A está a la derecha de B".

In [ ]:
# Procesamos la oración
oracion = 'A está a la derecha de B'.split()
tree1 = utils.parsear(oracion, parser_bloques)
tree1

Los anteriores comandos obtienen el árbol de análisis de la oración. No obstante, para obtener la fórmula podemos usar el siguiente comando:

In [ ]:
formula = utils.obtener_formula(oracion, parser_bloques)
print(formula)

<a class="anchor" id="ej5"></a>**Ejercicio 5:** 

([Anterior ejercicio](#ej4)) ([Próximo ejercicio](#ej6))

Modifique la gramática `reglas_bloques` para poder representar las siguientes oraciones:

* B está a la izquierda de C.
* C está encima de A.
* A está debajo de D.


In [ ]:
# Definimos la gramática para el mundo de los bloques
reglas_bloques = r"""
% start O
#############################
# Reglas gramaticales
#############################

# La oración es un bloque (BL) seguido de 'está' y un vector (VE)
O[SEM=<?ve(?b)>] -> BL[SEM=?b] 'está' VE[SEM=?ve]

# Un vector (VE) es una dirección (D) seguida de un bloque (BL)
VE[SEM=<?d(?b)>] -> D[SEM=?d] BL[SEM=?b]

#############################
# Reglas léxicas
#############################

BL[SEM=<bloque_A>] -> 'A'
BL[SEM=<bloque_B>] -> 'B'
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE C <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE D <<<<<<<<<<
D[SEM=<\y.(\x.DERECHA(x,y))>] -> 'a' 'la' 'derecha' 'de'
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE IZQUIERDA <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE ENCIMA <<<<<<<<<<
>>>>>>>>>>> AQUI LA REPRESENTACIÓN DE DEBAJO <<<<<<<<<<
"""

# Instanciamos el objeto
gramatica_bloques = FeatureGrammar.fromstring(reglas_bloques)

# Instanciamos el parser
parser_bloques = FeatureEarleyChartParser(gramatica_bloques)

In [ ]:
# Procesamos la oración
oracion = 'B está a la izquierda de C'.split()
formula = utils.obtener_formula(oracion, parser_bloques)
print(formula)

In [ ]:
# Procesamos la oración
oracion = 'C está encima de A'.split()
formula = utils.obtener_formula(oracion, parser_bloques)
print(formula)

In [ ]:
# Procesamos la oración
oracion = 'A está debajo de D'.split()
formula = utils.obtener_formula(oracion, parser_bloques)
print(formula)

---

## Representación de preguntas <a class="anchor" id="preguntas"></a>

([Volver al inicio](#inicio))

Es hora de pasar al tema de cómo verificar la comprensión que el sistema tiene del texto. Recordemos que esto se comprueba de manera indirecta, mediante preguntas. Así pues, debemos implementar una manera automática de responder preguntas. Para ello requerimos tanto de una representación de las preguntas como de una manera de responderlas. Comenzaremos por lo primero.

Las preguntas las representaremos mediante fórmulas abiertas. La idea es la siguiente. La respuesta a una pregunta factual es una entidad que hace verdadero un hecho. Por ejemplo, la pregunta ¿dónde está el gato? puede representarse mediante la fórmula `ESTAR(e) & SUJETO(e, gato) & EN(x)`. De esta manera, textos como "El gato está en el tapete", el cual se representa mediante `ESTAR(e) & SUJETO(e, gato) & EN(tapete)`, hacen evidente que la respuesta es la entidad `tapete`, la cual puede reemplazar a `x` en la pregunta y producir una oración verdadera. Veremos este método de solución de preguntas con más detalle en un momento.

Por ahora, para representar preguntas, requerimos el siguiente parser:

In [ ]:
parser_preguntas = load_parser('preguntas_bloques.fcfg', trace=0)

Observe que las preguntas de tipo factual pueden representarse como fórmulas abiertas:

In [ ]:
pregunta1_ = '¿ qué está encima de A ?'.split()
pregunta1 = utils.obtener_formula(pregunta1_, parser_preguntas)
print(pregunta1)

In [ ]:
pregunta2_ = '¿ dónde está A ?'.split()
pregunta2 = utils.obtener_formula(pregunta2_, parser_preguntas)
print(pregunta2)

Las variables libres de la fórmula que representa la pregunta tienen un tipo, el cual restringe los posibles candidatos de respuesta a la pregunta.

In [ ]:
def tipo_variable(var):
    if var.name in ['x']:
        return 'bloque'
    elif var.name in ['D']:
        return 'direccion'
    else:
        raise Exception(f'¡El tipo de la variable {var.name} es desconocido!')
    
for var in pregunta1.free():
    print(f'El tipo de la variable {var} de la pregunta 1 es: {tipo_variable(var)}')

print('')

for var in pregunta2.free():
    print(f'El tipo de la variable {var} de la pregunta 2 es: {tipo_variable(var)}')

---

## Resolución de preguntas <a class="anchor" id="answer"></a>

([Volver al inicio](#inicio))

La solución de preguntas sobre un texto usará el siguiente pipeline:

   1. Representamos el texto mediante una base de conocimiento.
   2. Representamos la pregunta tipo factual mediante una fórmula abierta.
   3. Encontramos una lista de candidatos.
   4. Por cada candidato, creamos una fórmula $\psi$.
   5. Verificamos si $\psi$ se obtiene a partir de la base de conocimiento.
   
Veámos el siguiente ejemplo en forma diagramática:
   
   <img src="./imagenes/pipeline.png" width="600"/>
   
Explicaremos cada uno de los pasos a continuación.

**Representación del texto mediante una base de conocimiento**

En este paso creamos una base de conocimiento, la cual alimentamos con el texto y con información de dominio. El texto debemos representarlo mediante fórmulas de la lógica proposicional y las incluimos en los hechos de la base. La información de dominio las representamos mediante reglas que incluimos en las reglas de la base. Explicaremos los pasos mediante el siguiente texto:

* Dominio: El mundo de los bloques.
* Texto: "A está a la izquierda de B. C está a la derecha de B. C está encima de D."



In [ ]:
texto = "A está a la izquierda de B. C está a la derecha de B. C está encima de D."
texto = [oracion for oracion in texto.split('.') if oracion != '']
formulas = []
# Por cada oración del texto
for oracion_ in texto:
    oracion = oracion_.split()
    formula = utils.obtener_formula(oracion, parser_bloques)
    formulas.append(formula)
bk = LPQuery([str(f) for f in formulas])
print(bk)

El texto menciona tanto las entidades como los predicados, los cuales podemos extraer de las fórmulas usando la clase `Modelo` del módulo `parseMod`:

In [ ]:
# Instaciamos la clase Modelo para obtener la lista de candidatos
M = Modelo()
M.poblar_con(Ytoria(formulas))
print(M)

**Representación de la pregunta y lista de candidatos**

Vamos a representar las preguntas:

    * ¿qué está a la izquierda de B?
    * ¿dónde está C?

In [ ]:
pregunta1_ = '¿ qué está a la izquierda de B ?'.split()
pregunta1 = utils.obtener_formula(pregunta1_, parser_preguntas)
print('La representación de la pregunta')
print('\n\t¿qué está a la izquierda de B?')
print('es:\n\t', pregunta1)
pregunta2_ = '¿ dónde está C ?'.split()
pregunta2 = utils.obtener_formula(pregunta2_, parser_preguntas)
print('La representación de la pregunta')
print('\n\t¿dónde está C?')
print('es:\n\t', pregunta2)

Ahora debemos crear una lista de fórmulas que corresponden a los candidatos. Para ello, echaremos mano del objeto `Modelo`, el cual tiene guardados los bloques y direcciones:

In [ ]:
print('Candidatos pregunta 1:')
candidatos_pregunta1 = {}
# Obtenemos la lista de bloques a partir del modelo
bloques = M.entidades["individuo"]
# Iteramos por cada bloque para producir una fórmula
for bloque in bloques:
    formula = lp.parse(fr'\x.{pregunta1}({bloque})').simplify()
    print(formula)
    # Guardamos la fórmula correspondiente al candidato
    candidatos_pregunta1[str(bloque)] = str(formula)   
print('')
print('Candidatos pregunta 2:')
candidatos_pregunta2 = {}
# Obtenemos los predicados a partir del modelo
direcciones = M.predicados
# Iteramos sobre la combinación de predicados y bloques
for direccion, bloque in product(direcciones, bloques):
    formula = lp.parse(fr'\D(\x.{pregunta2}({bloque}))({direccion})').simplify()
    print(formula)
    # Guardamos la fórmula correspondiente a la combinación
    candidatos_pregunta2[(str(direccion),str(bloque))] = str(formula)

**Verificación de implicación**

El último paso es verificar, por cada fórmula candidata $\psi$, si esta se sigue lógicamente de la fórmula $\phi$ que representa el texto. Para lograr esto, usaremos el método `ask()` de la base de conocimiento:

In [ ]:
print('Nuestra base de conocimiento es:')
print('')
print(bk)
print('='*10, 'Pregunta 1', '='*10)
print(f'\n\t {" ".join(pregunta1_)}')
for variables in candidatos_pregunta1.keys():
    formula_candidata = candidatos_pregunta1[variables]
    res = bk.ask(formula_candidata)
    if res == 'exito':
        print(f'\nRespuesta:\n\n\t{variables}')
print('\n\n')
print('='*10, 'Pregunta 2', '='*10)
print(f'\n\t {" ".join(pregunta2_)}')
for variables in candidatos_pregunta2.keys():
    formula_candidata = candidatos_pregunta2[variables]
    res = bk.ask(formula_candidata)
    if res == 'exito':
        print(f'\nRespuesta:\n\n\t{variables}')

<a class="anchor" id="ej6"></a>**Ejercicio 6:** 

([Anterior ejercicio](#ej5))

Incluya las reglas del notebook "Razomaniento automático" (y también aquellas reglas de dominio geométrico que sean relevantes) en la base de conocimiento para responder de manera automática las siguientes preguntas:

* ¿qué está a la derecha de A?
* ¿qué está debajo de C?
* ¿dónde está B?

Utilice la siguiente celda para comprobar su base de conocimiento.

In [ ]:
text_data = {\
    "id":"texto_bloques", \
    "texto": "A está a la izquierda de B. C está a la derecha de B. C está encima de D.", \
    "preguntas": ["¿qué está encima de D?",
                  "¿qué está a la derecha de B?"], \
    "respuestas": [[('bloque_C',)], 
                   [('bloque_C',), ('bloque_D',)]]\
}

pipeline_responder_preguntas(text_data=text_data, 
                             gramatica_texto=reglas_bloques, 
                             gramatica_preguntas='preguntas_bloques.fcfg',
                             bk=bk,
                             verbose=True)

## En este notebook usted aprendió

([Volver al inicio](#inicio))

Usar la librería `nltk` para:

* Representar fórmulas de la lógica de predicados.
* Representar fórmulas del cálculo $\lambda$.
* Implementar gramáticas lógicas.
* Combinar gramáticas lógicas y bases de conocimiento para responder preguntas.